In [15]:
import requests
year = 2021
key = 'c8763b1f3e686c707cf581321b8c16152e9d30e3'
url = f'https://api.census.gov/data/{year}/acs/acs5?get=NAME&for=place:*&in=state:*&key={key}'
r = requests.get(url)
print('resonse code:', r.status_code)
try:
    places = r.json()
except:
    print('failed to parse json')
    print(r.text)
print(places[:20])

resonse code: 200
[['NAME', 'state', 'place'], ['Abanda CDP, Alabama', '01', '00100'], ['Abbeville city, Alabama', '01', '00124'], ['Adamsville city, Alabama', '01', '00460'], ['Addison town, Alabama', '01', '00484'], ['Akron town, Alabama', '01', '00676'], ['Alabaster city, Alabama', '01', '00820'], ['Albertville city, Alabama', '01', '00988'], ['Alexander City city, Alabama', '01', '01132'], ['Alexandria CDP, Alabama', '01', '01180'], ['Aliceville city, Alabama', '01', '01228'], ['Allgood town, Alabama', '01', '01396'], ['Altoona town, Alabama', '01', '01660'], ['Andalusia city, Alabama', '01', '01708'], ['Anderson town, Alabama', '01', '01756'], ['Anniston city, Alabama', '01', '01852'], ['Arab city, Alabama', '01', '02116'], ['Ardmore town, Alabama', '01', '02260'], ['Argo town, Alabama', '01', '02320'], ['Ariton town, Alabama', '01', '02428']]


In [60]:
places[:2]

[['NAME', 'state', 'place'], ['Abanda CDP, Alabama', '01', '00100']]

In [50]:

import time
year = 2021

results = []
counter = Value('i',0)
    
def get_place(place):
    # print(place)
    global counter
    with counter.get_lock():
        counter.value += 1
    print(counter.value,'\n')
    year = 2021
    key = 'c8763b1f3e686c707cf581321b8c16152e9d30e3'
    p = place[2]
    s = place[1]
    url = f'https://api.census.gov/data/{year}/acs/acs5?get=NAME&for=county%20(or%20part):*&in=state:{s}%20place:{p}&key={key}'
    # print(url)
    done = False
    while not done:
        r = requests.get(url)
        if r.status_code != 200:
            print(r.status_code)
            time.sleep(.1)
            print(r.text)
            continue
        try:
            p_list = r.json()
            # results = results + places[1:]
            with open(f'threads/{current_process().pid}.dat','a') as f:
                for l in p_list[1:]:
                    f.write('|'.join(l) + '\n')
            done = True
        except Exception as e:
            print(e)
            print(r.text)


In [48]:
from multiprocessing import Pool, current_process, Value



# if __name__ == '__main__':
#     with Pool(50) as pool:
#         pool.map(get_place, places[1:])

In [52]:
import pandas as pd
import glob
import os



all_files = glob.glob("threads/*.dat")
print(all_files)

li = []

for filename in all_files:
    df = pd.read_csv(filename,names=["NAME","state","place","county"],sep='|')
    li.append(df)

frame = pd.concat(li)

frame.state = frame.state.apply(lambda x: str(x).zfill(2))
frame.county = frame.county.apply(lambda x: str(x).zfill(3))
frame.place = frame.place.apply(lambda x: str(x).zfill(5))
frame.head()

['threads/19701.dat', 'threads/19712.dat', 'threads/19706.dat', 'threads/19683.dat', 'threads/16406.dat', 'threads/19708.dat', 'threads/19709.dat', 'threads/19670.dat', 'threads/19686.dat', 'threads/19715.dat', 'threads/19707.dat', 'threads/19673.dat', 'threads/19674.dat', 'threads/19690.dat', 'threads/19672.dat', 'threads/19685.dat', 'threads/19669.dat', 'threads/19688.dat', 'threads/19668.dat', 'threads/19680.dat', 'threads/19693.dat', 'threads/19691.dat', 'threads/19667.dat', 'threads/19710.dat', 'threads/19713.dat', 'threads/19702.dat', 'threads/19679.dat', 'threads/19681.dat', 'threads/19705.dat', 'threads/19694.dat', 'threads/19699.dat', 'threads/19700.dat', 'threads/19716.dat', 'threads/19689.dat', 'threads/19687.dat', 'threads/19682.dat', 'threads/19714.dat', 'threads/19695.dat', 'threads/19677.dat', 'threads/19696.dat', 'threads/19692.dat', 'threads/19697.dat', 'threads/19704.dat', 'threads/19678.dat', 'threads/19711.dat', 'threads/19675.dat', 'threads/19676.dat', 'threads/196

,NAME,state,place,county
0,"Chatham County (part), Bloomingdale city, Georgia",13,08844,051
1,"Fannin County (part), Blue Ridge city, Georgia",13,08928,111
2,"Clay County (part), Bluffton town, Georgia",13,08956,061
3,"Burke County (part), Blythe city, Georgia",13,09040,033
4,"Richmond County (part), Blythe city, Georgia",13,09040,245


In [56]:
frame.to_csv('state-place-county.csv', index=False)

In [58]:
frame.groupby(['state','place','county']).count().sort_values('NAME',ascending=False)

NAME
state place county      
01    00100 017        1
39    54838 011        1
      54726 047        1
      54698 077        1
      54684 061        1
...                  ...
21    36865 111        1
      36604 037        1
      36460 113        1
      36374 111        1
72    88293 027        1

[33341 rows x 1 columns]

In [16]:
pulled = pd.DataFrame(places[1:],columns=places[0])
pulled.head()

,NAME,state,place
0,"Abanda CDP, Alabama",01,00100
1,"Abbeville city, Alabama",01,00124
2,"Adamsville city, Alabama",01,00460
3,"Addison town, Alabama",01,00484
4,"Akron town, Alabama",01,00676


In [53]:
missing = pulled.merge(frame,on=['state','place'],how='left')

In [54]:
tmp=missing[missing.county.isnull()][['NAME_x','state','place']]
m_list=list(tmp.itertuples(index=False, name=None))
print(m_list)

# for w in m_list:
#     get_place(w)

[]


In [32]:
len(pulled.groupby(['state','place']))

31908

In [33]:
len(frame.groupby(['state','place']))

31761